In [2]:

import os

import torch
# Load PhoBERT-base in fairseq
from fairseq.models.roberta import RobertaModel

phobert_base_folder = os.path.join("PhoBERT_base_fairseq")
bpe_codes_file = os.path.join(phobert_base_folder , 'bpe.codes')
phobert = RobertaModel.from_pretrained(phobert_base_folder, checkpoint_file='model.pt', bpe='fastbpe', bpe_codes=bpe_codes_file).eval()

2022-06-04 11:45:58 | INFO | fairseq.file_utils | loading archive file PhoBERT_base_fairseq
2022-06-04 11:45:58 | INFO | fairseq.tasks.masked_lm | dictionary: 64000 types


In [3]:
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."
subwords = phobert.encode(line)

last_layer_features = phobert.extract_features(subwords)
assert last_layer_features.size() == torch.Size([1, 9, 768])


all_layers = phobert.extract_features(subwords, return_all_hiddens=True)
assert len(all_layers) == 13
assert torch.all(all_layers[-1] == last_layer_features)

In [4]:
masked_line = 'Tôi là sinh_viên trường  <mask> ngoại_thương .'
topk_filled_outputs = phobert.fill_mask(masked_line, topk=5)
print(topk_filled_outputs)


[('Tôi là sinh_viên trường Đại_học ngoại_thương .', 0.36145657300949097, 'Đại_học'), ('Tôi là sinh_viên trường ĐH ngoại_thương .', 0.291090726852417, 'ĐH'), ('Tôi là sinh_viên trường đại_học ngoại_thương .', 0.2378615140914917, 'đại_học'), ('Tôi là sinh_viên trường Kinh_tế ngoại_thương .', 0.030656956136226654, 'Kinh_tế'), ('Tôi là sinh_viên trường Quản_lý ngoại_thương .', 0.02417973428964615, 'Quản_lý')]


In [5]:
phobert.eval()  # disable dropout (or leave in train mode to finetune

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=T

In [6]:


#Incorporate the BPE encoder into PhoBERT
tokens = phobert.encode('Tôn Ngộ Không phò Đường Tăng đi Tây Trúc thỉnh kinh')
print('tokens list : ', tokens)
# Decode ngược lại thành câu từ chuỗi index token
phobert.decode(tokens)  # 'Hello world!'

tokens list :  tensor([    0, 11623, 31433,   453, 44334,  2080,  5922,    57,   934,  8181,
        31686,  3078,     2])


'Tôn Ngộ Không phò Đường Tăng đi Tây Trúc thỉnh kinh'

In [7]:
last_layer_features = phobert.extract_features(tokens)
assert last_layer_features.size() == torch.Size([1, 13, 768])
assert tokens.size() == torch.Size([13])

In [8]:
all_layers = phobert.extract_features(tokens, return_all_hiddens=True)
print('Last layer features: ', all_layers[-1] == last_layer_features)


Last layer features:  tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])


In [9]:
from segment import WordSegmenter

segmenter = WordSegmenter()
text = 'Tôn Ngộ Không phò Đường Tăng đi thỉnh kinh tại Tây Trúc'
words = segmenter.segment_string_to_token(text)
for i, token in enumerate(words):
  if token == 'phò':
    words[i] = ' <mask>'

In [10]:
text_masked_tok = ' '.join(words)
text_masked_tok

'Tôn_Ngộ_Không  <mask> Đường Tăng đi thỉnh_kinh tại Tây Trúc'

In [11]:
from copy import copy

words1 = copy(words)

In [12]:
words1[8:9] = ["Tây_Trúc"]
words1

['Tôn_Ngộ_Không',
 ' <mask>',
 'Đường',
 'Tăng',
 'đi',
 'thỉnh_kinh',
 'tại',
 'Tây',
 'Tây_Trúc']

In [13]:
text_masked_tok = ' '.join(words)
text_masked_tok

'Tôn_Ngộ_Không  <mask> Đường Tăng đi thỉnh_kinh tại Tây Trúc'

In [14]:

import numpy as np


topk_filled_outputs = phobert.fill_mask(text_masked_tok, topk=10)
topk_probs = [item[1] for item in topk_filled_outputs]
print('Total probability: ', np.sum(topk_probs))
print('Input sequence: ', text_masked_tok)
print('Top 10 in mask: ')
for i, output in enumerate(topk_filled_outputs):
  print(output[0])

Total probability:  0.8698028372600675
Input sequence:  Tôn_Ngộ_Không  <mask> Đường Tăng đi thỉnh_kinh tại Tây Trúc
Top 10 in mask: 
Tôn_Ngộ_Không và Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không đưa Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không cõng Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không cùng Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không hộ_tống Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không theo Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không dẫn Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không chở Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không , Đường Tăng đi thỉnh_kinh tại Tây Trúc
Tôn_Ngộ_Không tháp_tùng Đường Tăng đi thỉnh_kinh tại Tây Trúc


In [15]:
doc = phobert.extract_features_aligned_to_words("học_sinh cấp 3 được đến trường sau nghỉ dịch covid")

AttributeError: type object 'EnglishDefaults' has no attribute 'create_tokenizer'

In [103]:
!pip install spacy

In [104]:
!pip install glob2

  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9321 sha256=932d61833b627b553798a1a082d47d8c3da8385606f00cc8e626ea832a26d18f
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\d7\3c\72\5300602ba1269ffce8cff5dcf7b525fee756b57455903c37ba
Successfully built glob2


In [16]:
import glob2

data_path = r'C:\Users\admin\Documents\code\VNTC\Data\10Topics\Ver1.1'
train_path = os.path.join(data_path, 'Train_Full', '*', '*.txt')
test_path = os.path.join(data_path, 'Test_Full', '*', '*.txt')

In [18]:
from tqdm import tqdm

def read_text(path):
  with open(path, 'r', encoding='utf-16') as f:
    data = f.read()
  return data

def make_data(path):
  texts = []
  labels = []
  for file_path in tqdm(glob2.glob(path)):
      try:
          content = read_text(file_path)
          label = file_path.split(os.sep)[-2]
          texts.append(content)
          labels.append(label)
      except:
          continue
  return texts, labels

text_train, label_train = make_data(train_path)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33759/33759 [01:37<00:00, 345.01it/s]


In [141]:
text_test, label_test = make_data(test_path)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 50373/50373 [02:31<00:00, 332.08it/s]


In [23]:
import pickle

def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)

def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj

# Lưu lại các files
# _save_pkl('.\\text_train.pkl', text_train)
# _save_pkl('.\\label_train.pkl', label_train)
# _save_pkl('.\\text_test.pkl', text_test)
# _save_pkl('.\\label_test.pkl', label_test)

In [24]:
text_train = _load_pkl('.\\text_train.pkl')
label_train = _load_pkl('.\\label_train.pkl')
text_test = _load_pkl('.\\text_test.pkl')
label_test = _load_pkl('.\\label_test.pkl')

In [25]:
text_train[0], label_train[0]

(' Thành lập dự án POLICY phòng chống HIV/AIDS ở VN (NLĐ)- Quỹ hỗ trợ khẩn cấp về AIDS của Hoa Kỳ vừa thành lập dự án POLICY tại VN với cam kết hỗ trợ Chính phủ và nhân dân VN đối phó HIV/AIDS.Dự án có nhiệm vụ chính là cải thiện công tác phòng chống HIV/AIDS thông qua các lĩnh vực xây dựng chính sách, rà soát các văn bản pháp luật, xây dựng chiến lược quảng bá, xây dựng chương trình đào tạo về phòng chống HIV/AIDS, lên kế hoạch bố trí nguồn lực, huấn luyện và nghiên cứu về phương tiện truyền thông đại chúng, tổ chức các hoạt động nhằm giảm kỳ thị và phân biệt đối xử đối với người có HIV/AIDS... Theo TTXVN, dự án POLICY đặc biệt quan tâm đến công tác truyền thông phòng chống HIV/AIDS, coi đây là một biện pháp tích cực và hữu hiệu trong việc phòng chống có hiệu quả HIV/AIDS. Thời gian tới, dự án POLICY sẽ tiếp tục tổ chức các hoạt động nhằm nâng cao nhận thức cho những người có trách nhiệm với công tác chỉ đạo phòng chống HIV/AIDS.\n\n',
 'Chinh tri Xa hoi')

In [26]:

max_sequence_length = 256
def convert_lines(lines, vocab, bpe):
    '''
    :param lines:
    :param vocab:
    :param bpe:
    :return:
    '''
    outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32)
    eos_id = 2
    pad_id = 1
    for idx, row in tqdm(enumerate(lines), total=len(lines)):
        subwords = bpe.encode('<s>' + row + '</s>')
        input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
        if len(input_ids) > max_sequence_length:
            input_ids = input_ids[:max_sequence_length]
            input_ids[-1] = eos_id
        else:
            input_ids = input_ids + [pad_id, ] * (max_sequence_length - len(input_ids))
        outputs[idx,: ] = np.array(input_ids, dtype=np.int32)
    return outputs

from fairseq.data import Dictionary

vocab = Dictionary()
vocab.add_from_file(os.path.join(phobert_base_folder, 'dict.txt'))
lines = ['Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19', 'số lượng ca nhiễm bệnh đã giảm bắt đầu từ tháng 5 nhờ biện pháp mạnh tay']
[x1, x2] = convert_lines(lines, vocab, phobert.bpe)
print('x1 tensor encode: {}, shape: {}'.format(x1[:10], x1.size))
print('x1 tensor decode: ', phobert.decode(torch.tensor(x1))[:103])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2000.62it/s]

x1 tensor encode: [63117  1302 61542  5958    11   915   222   537   933    39], shape: 256
x1 tensor decode:  <s>Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19</s> <pad> <pad> <pad> <pad> <pad> <


In [27]:
X = convert_lines(text_train, vocab, phobert.bpe)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33759/33759 [01:32<00:00, 365.90it/s]


In [33]:
X.shape

(33759, 256)

In [28]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(label_train)
y = lb.fit_transform(label_train)
print(lb.classes_)
print('Top 5 class índices', y[:5])

['Chinh tri Xa hoi' 'Doi song' 'Khoa hoc' 'Kinh doanh' 'Phap luat'
 'Suc khoe' 'The gioi' 'The thao' 'Van hoa' 'Vi tinh']
Top 5 class índices [0 0 0 0 0]


In [29]:
# _save_pkl(os.path.join('.', 'PhoBERT_pretrain','X1.pkl'), X)
# _save_pkl(os.path.join('.', 'PhoBERT_pretrain','y1.pkl'), y)
# _save_pkl(os.path.join('.', 'PhoBERT_pretrain','labelEncoder1.pkl'), lb)


X  = _load_pkl(os.path.join('.', 'PhoBERT_pretrain','X1.pkl'))
y  = _load_pkl(os.path.join('.', 'PhoBERT_pretrain','y1.pkl'))
print('length of X: ', len(X))
print('length of y: ', len(y))

length of X:  33759
length of y:  33759


In [30]:
from fairseq.models.roberta import RobertaModel

phoBERT_cls = phobert = RobertaModel.from_pretrained(phobert_base_folder, checkpoint_file='model.pt', bpe='fastbpe', bpe_codes=bpe_codes_file)
phoBERT_cls.eval()
phoBERT_cls.register_classification_head('new_task', num_classes=10)
tokens = 'Học_sinh được nghỉ học bắt đầu từ tháng 3 do ảnh hưởng của dịch covid-19'
token_idxs = phoBERT_cls.encode(tokens)
logprobs = phoBERT_cls.predict('new_task', token_idxs)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
logprobs

2022-06-04 13:15:12 | INFO | fairseq.file_utils | loading archive file PhoBERT_base_fairseq
2022-06-04 13:15:12 | INFO | fairseq.tasks.masked_lm | dictionary: 64000 types


tensor([[-1.9975, -2.3338, -2.3026, -2.2891, -2.4389, -2.4729, -2.4866, -2.3556,
         -2.3843, -2.0854]], grad_fn=<LogSoftmaxBackward0>)

In [31]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score

def evaluate(logits, targets):
    """
    Đánh giá model sử dụng cho acuracy và f1-score
    :param logits: (B,C) torch.LongTensor  giá trị predicted cho các class output
    :param targets: (B) torch.LongTensor actual target indices
    :return: acc
    f1_score
    """
    logits = logits.detach().cpu().numpy()
    y_pred = np.argmax(logits, axis=1)
    targets = targets.detach().cpu().numpy()
    f1 = f1_score(targets, y_pred, average='weighted')
    acc = accuracy_score(targets, y_pred)
    return acc, f1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logits = torch.tensor([[0.1, 0.2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)
targets = torch.tensor([1, 2, 2]).to(device)
evaluate(logits, targets)


(0.6666666666666666, 0.5333333333333333)

In [32]:
def validate(valid_loader, model, device):
    model.eval()
    accs = []
    f1s = []
    with torch.no_grad():
        for x_batch, y_batch in valid_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            output = model.predict('new_task', x_batch)
            logits = torch.exp(output)
            acc, f1 = evaluate(logits, y_batch)
            accs.append(acc)
            f1s.append(f1)
    mean_acc = np.mean(accs)
    mean_f1 = np.mean(f1s)
    return mean_acc, mean_f1


def train_on_epoch(train_loader, model, optimizer, epoch, num_epochs, criteria, device, log_aggr=100):
    model.train()
    sum_epoch_loss = 0
    sum_acc =0
    sum_f1 = 0
    start = time.time()
    for i, (x_batch, y_batch) in enumerate(train_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model.predict('new_task', x_batch)
        logits = torch.exp(y_pred)
        acc, f1 = evaluate(logits, y_batch)
        loss = criteria(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        loss_val = loss.item()
        sum_epoch_loss += loss_val
        sum_acc += acc
        sum_f1 += f1
        iter_num = epoch * len(train_loader) + i + 1
        if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f),  avg acc: %.4f, avg f1: %.4f, (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),  sum_acc/(i+1), sum_f1/(i+1),
                  len(x_batch) / (time.time() - start)))
        start = time.time()

In [ ]:
from torch import nn
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import time
import pickle
from tqdm import tqdm

from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold


EPOCHS = 20
BATCH_SIZE = 6
ACCUMULATION_STEPS = 5
FOLD = 4
LR = 0.0001
LR_DC_STEP = 80
LR_DC = 0.1
CUR_DIR = os.path.dirname(os.getcwd())
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FOLD = 4
CKPT_PATH2 = 'model_ckpt2'

if not os.path.exists(CKPT_PATH2):
    os.mkdir(CKPT_PATH2)

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X, y))
for fold, (train_idx, val_idx) in enumerate(splits):
    best_score = 0
    if fold != FOLD:
        continue
    print("Training for fold {}".format(fold))

    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X[train_idx], dtype=torch.long), torch.tensor(y[train_idx], dtype=torch.long))
    valid_dataset  = torch.utils.data.TensorDataset(torch.tensor(X[val_idx], dtype=torch.long), torch.tensor(y[val_idx], dtype=torch.long))

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader  = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

    MODEL_LAST_CKPT = os.path.join(CKPT_PATH2, "last_checkpoint.pth.tar")
    if os.path.exists(MODEL_LAST_CKPT):
        print('Load checkpoint model!')
        phoBERT_cls = torch.load(MODEL_LAST_CKPT)
    else:
        print("Load model pretrained")
        phoBERT_cls = RobertaModel.from_pretrained(phobert_base_folder, checkpoint_file='model.pt', bpe='fastbpe', bpe_codes=bpe_codes_file).eval()
        phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune
        phoBERT_cls.register_classification_head('new_task', num_classes=10)

    phoBERT_cls.to(DEVICE)
    param_optimizer = list(phoBERT_cls.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    print("Init Optimizer", "LayerNorm.bias", "LayerNorm.Weight")
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    num_train_optimization_steps = int(EPOCHS * len(train_dataset)/BATCH_SIZE/ACCUMULATION_STEPS)
    optimizer = AdamW(optimizer_grouped_parameters, lr=LR, correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)
    scheduler0 = get_constant_schedule(optimizer)

    criteria = nn.NLLLoss()
    avg_loss = 0
    avg_accuracy = 0
    frozen = True
    for epoch in tqdm(range(EPOCHS)):
        if epoch > 0 and frozen:
            for child in phoBERT_cls.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
            del scheduler0
            torch.cuda.empty_cache()
        print("Epoch: ", epoch)
        train_on_epoch(train_loader=train_loader,
                       device=DEVICE,
                       model=phoBERT_cls,
                       criteria=criteria,
                       optimizer=optimizer,
                       num_epochs=EPOCHS,
                       epoch=epoch,
                       log_aggr=100)
        if not frozen:
            scheduler.step()
        else:
            scheduler0.step()
        optimizer.zero_grad()
        acc, f1 = validate(valid_loader, phoBERT_cls, device=DEVICE)

        print('Epoch {} validation: acc: {:.4f}, f1: {:.4f} \n'.format(epoch, acc, f1))
        ckpt_dict = {
                    'epoch': epoch + 1,
                    'state_dict': phoBERT_cls.state_dict(),
                    'optimizer': optimizer.state_dict()
                }
        # Save model checkpoint into 'latest_checkpoint.pth.tar'
        torch.save(ckpt_dict, MODEL_LAST_CKPT)